# Generate AMITT page metadata

Create a page for each of the AMITT objects, if it doesn't already exist.  If it does, update the metadata on it, but take care to preserve any human-generated notes. 

In [1]:
import pandas as pd
import os

xlsx = pd.ExcelFile('amitt_metadata_v3.xlsx')
metadata = {}
for sheetname in xlsx.sheet_names:
    metadata[sheetname] = xlsx.parse(sheetname)
print('loaded {}'.format(metadata.keys()))

for entity in ['phase', 'tactic', 'technique', 'task', 'incident']:
    entities = entity + 's'
    entitydir = '../{}'.format(entities)
    if not os.path.exists(entitydir):
        os.makedirs(entitydir)
    
    template = open('template_{}.md'.format(entity)).read()
    dfentity = metadata[entities]
    for index, row in dfentity[dfentity['name'].notnull()].iterrows():
        
        with open('../{}/{}.md'.format(entities,row['id']), 'w') as f:
            if entity == 'phase':
                f.write(template.format(id=row['id'], name=row['name'],
                                        summary=row['summary']))
            if entity == 'tactic':
                f.write(template.format(id=row['id'], name=row['name'],
                                        phase=row['phase'], summary=row['summary']))            
            if entity == 'technique' or entity == 'task':
                f.write(template.format(id=row['id'], name=row['name'],
                                        tactic=row['tactic'], summary=row['summary']))
            if entity == 'incident':
                f.write(template.format(id=row['id'], name=row['name'],
                                        type=row['type'], summary=row['summary'],
                                        yearstarted=row['Year Started'], 
                                        fromcountry=row['From country'],
                                        tocountry=row['To country'],
                                        foundvia=row['Found via'],
                                        dateadded=row['When added'],
                                        techniques=''))
            f.close()

loaded dict_keys(['phases', 'tactics', 'techniques', 'tasks', 'incidents', 'incidenttechniques'])


In [12]:
# todo: protect anything under the "dO NOt WRITE ABOVE THIS" line
# todo: create the matrix (clickable?)
# todo: add in the existing incident pages
# todo: add in the existing tactic notes from spreadsheets
# todo: add all framework comments to the repo issues list
# todo: add the tracking of incidents to techniques (use the incidenttechiques field)

In [12]:
metadata['incidenttechniques']

,id,name,incident,technique,summary
0,I00002T001,buy FB targeted ads,I00002,T0018,NaN
1,I00002T002,"Promote ""funding"" campaign",I00002,T0017,NaN
2,I00002T003,create web-site - information pollution,I00002,"T0019,T0056",NaN
3,I00002T004,"SEO optimisation/manipulation (""key words"")",I00002,T0046,NaN
4,I00002T005,legacy web content,I00002,T0058,NaN
5,I00002T006,hard to remove content and/or campaign/exploit...,I00002,T0058,NaN
6,I00005T001,Twitter bots amplify & manipulate,I00005,T0054,NaN
7,I00005T002,Twitter trolls amplify & manipulate,I00005,T0053,NaN
8,I00005T003,Targeted FB paid ads,I00005,T0018,NaN
9,I00005T004,Fake FB groups + dark content,I00005,T0007,NaN
